### All imports needed for notebook

In [ ]:
import os
import pickle as pk
import pandas as pd
import os

# -- local files
from modules import spotify_methods as sm
from modules import content_recommender as cr
from modules import recommender_methods as rm

# 1) Load Model and Training Data

In [ ]:
loaded = rm.load_model()

In [ ]:
filename = os.getcwd() + "\\model" + "\\training_data.pkl"

with open(filename, 'rb') as fp:
    training_data = pk.load(fp)


# 2) Select a playlist and no of recommendations (max 1000)

In [ ]:
select_pid = 100000
required_recommendations = 1000

In [ ]:
scores, all_rec_songs = loaded([str(select_pid)])
print(f"Recommendations subset: {all_rec_songs[0][:10]}")

# 3) Remove songs that are already in the playlist

In [ ]:
orig_playlist = rm.show_playlist(select_pid, training_data)

In [ ]:
selected_recommendations = rm.decode_and_select(all_rec_songs, required_recommendations)

In [ ]:
de_duped_rec = rm.remove_known_positives(selected_recommendations, orig_playlist)

# 4) Use a Content Based Recommendor to sort through recommendation results

In [ ]:
cached_songs = pd.read_csv('./song_features.csv')
to_cache_items=[]

In [ ]:
sr_features = []
for sr in de_duped_rec:

    search_result, to_cache = sm.get_song_data(sr, cached_songs)
    if(search_result is not None):
        sr_features += [search_result]
    if(search_result is None):
        print(f'''Not found: {sr}''')
    if(to_cache):
        to_cache_items += [search_result]


In [ ]:
op_features = []
for op in orig_playlist:
    
    search_result, to_cache = sm.get_song_data(op, cached_songs)
    if(search_result is not None):
        op_features += [search_result]
    if(search_result is None):
        print(f'''Not found: {op}''')
    if(to_cache):
        to_cache_items += [search_result]

In [ ]:
new_cached_songs = pd.DataFrame(to_cache_items)
combine_frames = [cached_songs, new_cached_songs]

all_cached_songs = pd.concat(combine_frames)
all_cached_songs.to_csv(path_or_buf="song_features.csv", index=False)

In [ ]:
song_cluster_pipeline, number_cols = cr.cluster_pipeline(sr_features, 20)

In [ ]:
cr.recommend_songs(op_features, sr_features, song_cluster_pipeline, number_cols)